# Evaluation NER - échantillon tiré d'une transcription manuelle - segmentation par règle des phrases

## Imports

In [1]:
import re

from bs4 import BeautifulSoup
import spacy
import srsly

# Fonction d'ouverture des fichiers

In [2]:
def open_file(source):
    '''open a file and parsed its content with BeautifulSoup'''
    with open(source, 'r', encoding='utf8') as fh:
        file_content = fh.read()
        
    return file_content

# Création de la vérité de terrain NER

La vérité de terrain NER a été produite à l'aide de [Doccano](https://github.com/doccano/doccano). Le texte utilisé pour constituer une vérité de terrain est identique à celui utilisé pour la prédiction de NER. 

Cette méthodologie de test est plus adapté à l'évaluation d'un modèle entraîné pour une tâche spécifique. Cependant, il a été jugé pertinent de comparer ce qu'il était obtenu avec l'outil de NER brut, et ce qu'on attendrait de cet outil brut **au mieux** : la vérité de terrain. 

Le modèle de langue français utilisé avec spaCy permet de repérer 4 types d'entités : 
* `PER` : personne
* `LOC` : localisation
* `ORG` : organisations
* `MISC` : une entité générique indiquant une entité nommée, sans toutefois l'identifier.

Pour annoter le document, le fichier texte de l'échantillon testé a été annoté dans Doccano.

La composition d'une vérité de terrain sur un échantillon en sortie d'HTR brut est particulier, car demandant d'annoter des mots bruités, cela s'apparente, à certains moments, presque à lire une autre langue.

Il a été décidé de se placer au niveau des tokens, et d'annoter si celui-ci aurait dû être détecté. Par exemple, les noms de personne arrivant systématiquement au début d'une rangée dans le document originel. 

Les annotations manuelles respectent deux règles : 

* Les titres de civilités n'ont pas été relevés. 
* Les adresses ont été annotes dans leur forme complète, en incluant donc le mot "rue".

## Segmentation des phrases par règle

 On charge le [modèle de langue française](https://spacy.io/models/fr#fr_core_news_lg) de spaCy.

On lui ajoute une configuration personnalisée pour segmenter les phrases à l'aide d'une règle. On lui indique que chaque fin de phrase est marqué par un **point-virgule**.

In [5]:
nlp = spacy.load("fr_core_news_lg")
# On crée une variable config qui va stocker la configuration du sentencizer.
# La variable prend la forme d'un dictionnaire, contenant une clé punt_chars et un dictionnaire en valeur.
config = {"punct_chars": [";"]}

# On ajoute le sentencizer dans la pipeline, avec la config, et on s'assure qu'on fait exécuter cette tâche avant le parser.
nlp.add_pipe("sentencizer", config=config, before="parser")

# Traitement de l'échantillon testé

On récupère le texte qui servira à produire une prédiction pour le NER avec spaCy depuis un fichier texte. Ce dernier a été manuellement annoté avec des **points-virgules** pour marquer un changement d'unité sémantique, c'est-à-dire une rangée dans le tableau du document originel provenant du projet LECTAUREP.

In [8]:
sample_eval = open_file("../../corpus_test/lectaurep/doc_28_sample/manual_transcription/artificial_sentence_segmentation/text_with_artificial_separation.txt")

On passe le texte dans la pipeline NLP.

In [9]:
doc = nlp(sample_eval)

# Traitement de la vérité de terrain

On récupère la vérité de terrain dans un dictionnaire sous le format suivant :

```
{entities: [(start, end, type)]}
```

In [10]:
filepath = '../../corpus_test/lectaurep/doc_28_sample/manual_transcription/artificial_sentence_segmentation/ner_ground_truth/gt_artificial_sentence_segmentation.jsonl'
annotations_artificial_sentence_segmentation = list(srsly.read_jsonl(filepath))

annotations_artificial_sentence_segmentation

[{'id': 11,
  'data': 'An 1901 , mois de Février ;\nDupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme ;\n- d° - (par Mme) s. n. à son mari ;\nMurel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adresse (Séparation de biens) ;\nDubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,\nVve de Jules) et autres, à Paris, rue de l’Arcade 20 ;\nRouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir ;\nChabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de \nVersailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88 ;\nGauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère ;\nCusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père ;\nLecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre\nAuguste) décédé en son d

In [14]:
gr_data = {}
for annotation in annotations_artificial_sentence_segmentation :
    entities = [(start, end, type_) for start, end, type_ in annotation.get('label')]
    gt_data = {'entities': entities}
    text_gt = annotation.get('data')

gt_data

{'entities': [(0, 7, 'MISC'),
  (10, 25, 'MISC'),
  (28, 34, 'PER'),
  (41, 47, 'PER'),
  (51, 56, 'LOC'),
  (58, 70, 'LOC'),
  (74, 90, 'PER'),
  (139, 144, 'PER'),
  (150, 168, 'PER'),
  (175, 180, 'LOC'),
  (182, 200, 'LOC'),
  (205, 228, 'PER'),
  (235, 240, 'LOC'),
  (279, 285, 'PER'),
  (329, 338, 'PER'),
  (340, 353, 'PER'),
  (362, 367, 'PER'),
  (382, 387, 'LOC'),
  (389, 407, 'LOC'),
  (410, 414, 'PER'),
  (420, 435, 'PER'),
  (443, 448, 'LOC'),
  (450, 466, 'LOC'),
  (507, 514, 'PER'),
  (520, 540, 'PER'),
  (548, 567, 'PER'),
  (574, 585, 'LOC'),
  (587, 612, 'LOC'),
  (616, 629, 'PER'),
  (633, 650, 'PER'),
  (657, 668, 'LOC'),
  (670, 686, 'LOC'),
  (703, 711, 'PER'),
  (717, 734, 'PER'),
  (738, 743, 'LOC'),
  (745, 763, 'LOC'),
  (786, 792, 'PER'),
  (798, 817, 'PER'),
  (824, 829, 'LOC'),
  (831, 846, 'LOC'),
  (861, 871, 'PER'),
  (886, 911, 'ORG'),
  (933, 947, 'PER'),
  (974, 979, 'LOC'),
  (981, 997, 'LOC'),
  (1017, 1025, 'PER'),
  (1045, 1056, 'PER'),
  (1079, 11

## Vérification de l'alignement entre l'échantillon testé pour la prédiction NER et le texte utilisé pour la vérité de terrain NER

Pour vérifier l'alignement des deux textes utilisés pour la comparaison, on utilise un assert.

In [15]:
assert text_gt == sample_eval

In [16]:
# On teste les tokens
assert [t.text for t in nlp(text_gt)] == [t.text for t in doc]

# Evaluation

On crée un objet Example contenant normalement des informations pour l'entraînement. Il sera utilisé pour stocker le document traité par la pipeline NLP et donc les prédictions de NER, ainsi que la vérité de terrain. 

Voir :
* https://spacy.io/api/example
* https://spacy.io/api/data-formats#dict-input
* https://spacy.io/api/scorer#score_spans

In [17]:
from spacy.training import Example

example = Example.from_dict(doc, gt_data)

/home/hugoscheithauer/anaconda3/envs/ner_spacy/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "An 1901 , mois de Février ;
Dupuis, (par Pierre) à..." with entities "[(0, 7, 'MISC'), (10, 25, 'MISC'), (28, 34, 'PER')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


On instancie l'objet Scorer.

In [18]:
from spacy.scorer import Scorer

scorer = Scorer()
# On récupère les scores.
scores = scorer.score_spans([example], 'ents')

In [19]:
scores

{'ents_p': 0.6058823529411764,
 'ents_r': 0.6242424242424243,
 'ents_f': 0.6149253731343284,
 'ents_per_type': {'MISC': {'p': 0.06666666666666667,
   'r': 0.5,
   'f': 0.11764705882352941},
  'ORG': {'p': 0.42857142857142855,
   'r': 0.42857142857142855,
   'f': 0.42857142857142855},
  'LOC': {'p': 0.5277777777777778,
   'r': 0.5428571428571428,
   'f': 0.5352112676056338},
  'PER': {'p': 0.8026315789473685,
   'r': 0.7093023255813954,
   'f': 0.7530864197530864}}}

On peut procéder autrement avec :

In [68]:
from spacy.language import Language

scores_alt = nlp.evaluate([example])

scores_alt

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'pos_acc': None,
 'morph_acc': None,
 'morph_per_feat': None,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.6167664670658682,
 'ents_r': 0.6242424242424243,
 'ents_f': 0.6204819277108434,
 'ents_per_type': {'MISC': {'p': 0.06666666666666667,
   'r': 0.5,
   'f': 0.11764705882352941},
  'ORG': {'p': 0.42857142857142855,
   'r': 0.42857142857142855,
   'f': 0.42857142857142855},
  'LOC': {'p': 0.5507246376811594,
   'r': 0.5428571428571428,
   'f': 0.5467625899280575},
  'PER': {'p': 0.8026315789473685,
   'r': 0.7093023255813954,
   'f': 0.7530864197530864}},
 'tag_acc': None,
 'lemma_acc': None,
 'speed': 7004.694350005305}

In [70]:
len_gr_entities = len(gr_data['entities'])
len_prediction_entities = len(doc.ents)
print(f"nombre d'entités dans la vérité de terrain : {len_gr_entities} - nombre d'entités dans la prédiction : {len_prediction_entities}")

nombre d'entités dans la vérité de terrain : 168 - nombre d'entités dans la prédiction : 170


### Analyses

La segmentation des phrases par règles, reconstituant la structure logique du document, semble rendre le modèle générique légèrement plus performant.

# Visualisation de la prédiction NER

In [69]:
from spacy import displacy

html = displacy.render([doc], style='ent', page=True)